### Cell 0 — Repository Bootstrap & Experiment Registry (Required)

This cell ensures that the project repository is discoverable by Python **and**
defines the registry of experiments that will be executed in this notebook.

---

#### Why this is needed

- The notebook lives inside the `notebooks/` directory  
- Python does not automatically know where the project root is  
- All project code lives under the `src/` directory  
- Multiple experimental configurations (prompt-only, USE + map, OpenAI + map)
  must be executed in a single, reproducible workflow  

---

#### What this cell does

- Walks up the directory tree starting from the current notebook location  
- Finds the repository root (identified by the presence of a `src/` folder)  
- Adds that directory to `sys.path` so imports such as  
  `from src.config import ...` work correctly  
- Defines a central **experiment registry** describing:
  - which feature representation is used
  - where training data is read from
  - where trained models and metadata are saved  

---

#### Design principles

- Executed once at the very top of the notebook  
- Contains no learning or model-specific logic  
- Provides a single source of truth for all experiment configurations  
- Enables fair and controlled comparison between different feature setups  

---

This cell must be executed **before any imports from `src.*`**.  
All subsequent cells rely on the repository path and experiment definitions
established here.


In [1]:
# --- Cell 0: bootstrap so "src" is importable + run-all-experiments config ---

import sys
from pathlib import Path

# -------------------------------------------------
# Make repo root importable (find folder containing "src")
# -------------------------------------------------
p = Path.cwd().resolve()
for candidate in [p, *p.parents]:
    if (candidate / "src").is_dir():
        if str(candidate) not in sys.path:
            sys.path.insert(0, str(candidate))
        REPO_ROOT = candidate
        print("Repo root:", candidate)
        break
else:
    raise RuntimeError("Could not find repo root (no 'src' folder found in parents).")

# -------------------------------------------------
# Experiment registry (ALL experiments will run)
# -------------------------------------------------
DATA_DIR = REPO_ROOT / "data"

EXPERIMENTS = {
    "prompt_only": {
        "train_out": DATA_DIR / "output" / "train_out_prompt_only",
        "model_out": DATA_DIR / "output" / "models" / "exp_prompt_only",
        "feature_mode": "prompt_only",  # use X_prompt.npy
        "prompt_encoder_kind": "dan",
    },
    "use_map": {
        "train_out": DATA_DIR / "output" / "train_out_use",
        "model_out": DATA_DIR / "output" / "models" / "exp_use_map",
        "feature_mode": "fused",        # use X_concat.npy
        "prompt_encoder_kind": "dan",
    },
    "openai_map": {
        "train_out": DATA_DIR / "output" / "train_out_openai",
        "model_out": DATA_DIR / "output" / "models" / "exp_openai_map",
        "feature_mode": "fused",        # use X_concat.npy
        "prompt_encoder_kind": "openai-small",
    },
}

# Create output folders
for k, cfg in EXPERIMENTS.items():
    cfg["model_out"].mkdir(parents=True, exist_ok=True)

print("🧪 Will run experiments:", ", ".join(EXPERIMENTS.keys()))
for k, cfg in EXPERIMENTS.items():
    print(f" - {k:12s} | train_out={cfg['train_out']} | model_out={cfg['model_out']}")


Repo root: /Users/amirdonyadide/Documents/GitHub/Thesis
🧪 Will run experiments: prompt_only, use_map, openai_map
 - prompt_only  | train_out=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_prompt_only | model_out=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_prompt_only
 - use_map      | train_out=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_use | model_out=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_use_map
 - openai_map   | train_out=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_openai | model_out=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_openai_map


### Cell 1 — Experiment Setup & Global Configuration

This cell initializes the experiment environment and loads the global configuration required
for training and evaluation. It is designed so the notebook can run **all experiment
configurations in one pass** (prompt-only, USE + map, OpenAI + map) without manual edits.

**What this cell does:**

- **Loads global project configuration**
  - Paths (`PATHS`)
  - Runtime settings (`CFG`)
  - Operator groups (`DISTANCE_OPS`, `AREA_OPS`)
  - Dynamic extent configuration flags (`USE_DYNAMIC_EXTENT_REFS`, `ALLOW_FALLBACK_EXTENT`)
  - Extent reference column names (`EXTENT_DIAG_COL`, `EXTENT_AREA_COL`)

- **Defines a central experiment registry**
  - `EXPERIMENTS` — a dictionary where each experiment specifies:
    - `feature_mode`:
      - `prompt_only` (uses prompt embeddings only)
      - `fused` (uses concatenated map + prompt embeddings)
    - `train_out` — where the prepared matrices (`X_*`) and `train_pairs.parquet` are read from
    - `model_out` — where trained artifacts are saved

- **Sets embedding dimensions**
  - `MAP_DIM`, `PROMPT_DIM`
  - `FUSED_DIM = MAP_DIM + PROMPT_DIM`
  - The effective input dimension is derived per experiment:
    - `prompt_only` → `PROMPT_DIM`
    - `fused` → `FUSED_DIM`

- **Validates experiment folders**
  - Ensures each experiment’s `train_out` and `model_out` directories exist
  - Performs basic schema checks (required keys, valid feature modes)

**Design principles**

- Executed once near the top of the notebook (before data loading/training)
- Contains no model training logic
- Provides a single source of truth for experiment configuration
- Prevents accidental overwrites by saving each experiment into its own output folder

This cell must be executed **before any training or evaluation cells**. All experiment
comparisons depend on the consistent configuration established here.


In [2]:
# ===================== CELL 1 — PARAMETERS =====================

from pathlib import Path
import joblib

from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

from src.config import (
    PATHS, CFG, print_summary,
    DISTANCE_OPS, AREA_OPS,
    USE_DYNAMIC_EXTENT_REFS, ALLOW_FALLBACK_EXTENT,
    EXTENT_DIAG_COL, EXTENT_AREA_COL,
)

# -------------------------------------------------
# Global config overview (same for all experiments)
# -------------------------------------------------
print_summary()
print("USE_DYNAMIC_EXTENT_REFS:", USE_DYNAMIC_EXTENT_REFS)
print("ALLOW_FALLBACK_EXTENT  :", ALLOW_FALLBACK_EXTENT)
print("EXTENT_DIAG_COL:", EXTENT_DIAG_COL, " EXTENT_AREA_COL:", EXTENT_AREA_COL)

# -------------------------------------------------
# Dimensions from config (authoritative)
# -------------------------------------------------
MAP_DIM = int(CFG.MAP_DIM)
PROMPT_DIM = int(CFG.PROMPT_DIM)
FUSED_DIM = MAP_DIM + PROMPT_DIM
BATCH_SIZE = int(CFG.BATCH_SIZE)

print("MAP_DIM:", MAP_DIM)
print("PROMPT_DIM:", PROMPT_DIM)
print("FUSED_DIM:", FUSED_DIM)
print("BATCH_SIZE:", BATCH_SIZE)

# -------------------------------------------------
# Validate experiment registry (from Cell 0)
# -------------------------------------------------
required_keys = {"train_out", "model_out", "feature_mode"}

for exp_name, cfg in EXPERIMENTS.items():
    missing = required_keys - set(cfg.keys())
    if missing:
        raise ValueError(f"Experiment '{exp_name}' is missing keys: {missing}")

    cfg["train_out"] = Path(cfg["train_out"])
    cfg["model_out"] = Path(cfg["model_out"])

    cfg["train_out"].mkdir(parents=True, exist_ok=True)
    cfg["model_out"].mkdir(parents=True, exist_ok=True)

print("\n🧪 Experiments to be executed:")
for exp_name, cfg in EXPERIMENTS.items():
    print(
        f" - {exp_name:12s} | "
        f"feature_mode={cfg['feature_mode']:12s} | "
        f"train_out={cfg['train_out'].name} | "
        f"model_out={cfg['model_out'].name}"
    )

# -------------------------------------------------
# Helper: feature dimensions per experiment
# -------------------------------------------------
def get_feature_dims(feature_mode: str):
    """
    Returns (map_dim, prompt_dim, fused_dim) for a given feature mode.
    """
    if feature_mode == "prompt_only":
        return 0, PROMPT_DIM, PROMPT_DIM
    if feature_mode == "fused":
        return MAP_DIM, PROMPT_DIM, FUSED_DIM
    raise ValueError(f"Unknown feature_mode: {feature_mode}")


=== CONFIG SUMMARY ===
PROJ_ROOT  : /Users/amirdonyadide/Documents/GitHub/Thesis
DATA_DIR   : /Users/amirdonyadide/Documents/GitHub/Thesis/data
INPUT_DIR  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/input
OUTPUT_DIR : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output
MAPS_ROOT  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/samples/pairs
INPUT PAT. : *_input.geojson
--- User Study ---
USER_STUDY_XLSX : /Users/amirdonyadide/Documents/GitHub/Thesis/data/userstudy/UserStudy.xlsx
RESPONSES_SHEET : Responses
TILE_ID_COL     : tile_id
COMPLETE_COL    : complete
REMOVE_COL      : remove
TEXT_COL        : cleaned_text
PARAM_VALUE_COL : param_value
OPERATOR_COL    : operator
INTENSITY_COL   : intensity
--- Filters / IDs / Split ---
ONLY_COMPLETE   : True
EXCLUDE_REMOVED : True
PROMPT_ID       : r{i:08d}
SPLIT_BY        : tile
--- Outputs ---
PROMPT_OUT : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out
MAP_OUT    : /Users/amirdonyadide/Documents/GitH

## Step 2 — Prompt Embedding Generation (All Experiments)

In this step, we generate vector embeddings for all user prompts for **each experiment
configuration** (e.g., prompt-only, USE + map, OpenAI + map). The embedding backend is
selected per experiment via `PROMPT_ENCODER` (e.g., **USE-DAN/Transformer** or **OpenAI
text-embedding-3-* models**).

### What happens here?

For each entry in the experiment registry:

- Prompts are loaded from the user study source file.
- Only valid prompts are kept (`complete == True`, `remove == False`).
- The prompt embedding model is selected from the experiment configuration
  (mapped to `CFG.PROMPT_ENCODER`).
- All prompts are embedded in batches (with optional L2 normalization).
- The resulting embeddings and metadata are saved to an **experiment-specific folder**
  so no configuration overwrites another.

**Outputs per experiment** (written under `PATHS.PROMPT_OUT/<experiment_name>/`):

- `prompts_embeddings.npz` — matrix `E` and `ids`
- `prompts.parquet` — prompt_id, text, tile_id
- `meta.json` — model label, dimensionality, and export info

### Why this is encapsulated in a helper

To keep the notebook clean and reproducible, all logic related to:

- loading and filtering prompts,
- selecting the embedding backend,
- batching and normalization,
- saving outputs in a consistent format,

is encapsulated in `src/train/run_prompt_embeddings.py`.

The notebook only *orchestrates* experiments by calling this helper with an
experiment-specific configuration.

This design ensures:

- consistent prompt embeddings across training and evaluation,
- easy comparison between USE and OpenAI backends,
- clean separation between experiment orchestration (notebook) and implementation (src),
- safe parallel storage of artifacts for multiple experiment runs.


In [3]:
# ===================== CELL 2 — Prompt embeddings (experiment-scoped) =====================

from pathlib import Path
from dataclasses import replace

from src.train.run_prompt_embeddings import run_prompt_embeddings_from_config

print("\n=== Running prompt embeddings for all experiments ===")

prompt_meta_by_experiment = {}

for exp_name, cfg_exp in EXPERIMENTS.items():

    # IMPORTANT: cfg.PROMPT_ENCODER must be one of:
    #   "dan", "transformer", "openai-small", "openai-large"
    # (matches src/mapvec/prompts/prompt_embeddings.py)
    prompt_encoder_kind = cfg_exp.get("prompt_encoder_kind", CFG.PROMPT_ENCODER)

    # Create an experiment-specific cfg (CFG is frozen → use replace)
    CFG_EXP = replace(CFG, PROMPT_ENCODER=prompt_encoder_kind)

    prompt_out_dir = Path(PATHS.PROMPT_OUT) / exp_name
    prompt_out_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n🧪 Experiment: {exp_name}")
    print(f"   PROMPT_ENCODER: {CFG_EXP.PROMPT_ENCODER}")
    print(f"   Output dir    : {prompt_out_dir}")

    meta = run_prompt_embeddings_from_config(
        input_path=Path(PATHS.USER_STUDY_XLSX),
        out_dir=prompt_out_dir,
        cfg=CFG_EXP,
        paths=PATHS,
        verbosity=1,
        l2_normalize=True,
        also_save_embeddings_csv=False,
    )

    prompt_meta_by_experiment[exp_name] = meta
    print("   ✅ Prompt embeddings completed.")

print("\n✅ All prompt embeddings finished.")

2026-01-28 22:03:51 | INFO | Reading Excel: /Users/amirdonyadide/Documents/GitHub/Thesis/data/userstudy/UserStudy.xlsx (sheet=Responses)



=== Running prompt embeddings for all experiments ===

🧪 Experiment: prompt_only
   PROMPT_ENCODER: dan
   Output dir    : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/prompt_only


2026-01-28 22:03:51 | INFO | Filtered Excel rows: 786 → 562 (only_complete=True, exclude_removed=True)
2026-01-28 22:03:51 | INFO | Using local USE-dan at /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/model_dan
2026-01-28 22:03:51 | INFO | Loading USE-dan from local path: /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/model_dan …
2026-01-28 22:03:54 | INFO | Fingerprint not found. Saved model loading will continue.
2026-01-28 22:03:54 | INFO | path_and_singleprint metric could not be logged. Saved model loading will continue.
2026-01-28 22:03:54 | INFO | USE model loaded in 2.96s
2026-01-28 22:03:54 | INFO | Embedding 562 prompts with USE (batch_size=512, l2=True)…
2026-01-28 22:03:54 | INFO | Done USE embedding in 0.14s (dim=512).
2026-01-28 22:03:54 | INFO | Writing outputs to /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/prompt_only
2026-01-28 22:03:54 | INFO |   saved prompts_embeddings.npz (shape=(562, 512))
2026-01-28 22:03:54 | INFO |   s

   ✅ Prompt embeddings completed.

🧪 Experiment: use_map
   PROMPT_ENCODER: dan
   Output dir    : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/use_map


2026-01-28 22:03:58 | INFO | Fingerprint not found. Saved model loading will continue.
2026-01-28 22:03:58 | INFO | path_and_singleprint metric could not be logged. Saved model loading will continue.
2026-01-28 22:03:58 | INFO | USE model loaded in 3.57s
2026-01-28 22:03:58 | INFO | Embedding 562 prompts with USE (batch_size=512, l2=True)…
2026-01-28 22:03:58 | INFO | Done USE embedding in 0.15s (dim=512).
2026-01-28 22:03:58 | INFO | Writing outputs to /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/use_map
2026-01-28 22:03:58 | INFO |   saved prompts_embeddings.npz (shape=(562, 512))
2026-01-28 22:03:58 | INFO |   saved prompts.parquet (rows=562)
2026-01-28 22:03:58 | INFO |   saved meta.json
2026-01-28 22:03:58 | INFO | Reading Excel: /Users/amirdonyadide/Documents/GitHub/Thesis/data/userstudy/UserStudy.xlsx (sheet=Responses)
2026-01-28 22:03:58 | INFO | Filtered Excel rows: 786 → 562 (only_complete=True, exclude_removed=True)


   ✅ Prompt embeddings completed.

🧪 Experiment: openai_map
   PROMPT_ENCODER: openai-small
   Output dir    : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/openai_map


2026-01-28 22:03:59 | INFO | Embedding 562 prompts with OpenAI model=text-embedding-3-small (batch_size=512, l2=True)…
2026-01-28 22:04:01 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-01-28 22:04:03 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-01-28 22:04:03 | INFO | Done OpenAI embedding in 4.75s (dim=1536).
2026-01-28 22:04:03 | INFO | Writing outputs to /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/openai_map
2026-01-28 22:04:03 | INFO |   saved prompts_embeddings.npz (shape=(562, 1536))
2026-01-28 22:04:03 | INFO |   saved prompts.parquet (rows=562)
2026-01-28 22:04:03 | INFO |   saved meta.json


   ✅ Prompt embeddings completed.

✅ All prompt embeddings finished.


## Step 3 — Map Embeddings (Dynamic, Shared Across Experiments)

In this step, we compute **map embeddings** for all GeoJSON tiles that are eligible for the user study.  
Because map embeddings depend only on the input map data (not on the prompt encoder), they are computed **once** and stored in a **shared output folder**, then reused across all experiments.

### What this step does

1. **Filters tiles using the user study Excel**
   - Keeps only rows marked as *complete*
   - Excludes rows marked as *removed*
   - Extracts the set of allowed `tile_id`s (used to select which map folders to embed)

2. **Discovers and embeds GeoJSON maps**
   - Finds all GeoJSON files under `PATHS.MAPS_ROOT`
   - Keeps only those whose `map_id` is in the allowed set
   - Counts valid polygons per map to determine a dataset-wide `max_polygons`
     (used to normalize the `poly_count` feature safely)

3. **Computes map embeddings**
   - Uses `norm="extent"` for **dynamic per-map normalization**
   - Ensures all vectors have consistent dimensionality
   - Skips maps with invalid geometries or degenerate extents

4. **Stores dynamic extent references (required for parameter scaling)**
   - `extent_diag_m`
   - `extent_area_m2`

   These are saved alongside embeddings and are later used to convert
   normalized parameters (`param_norm`) into real-world units:
   - distance operators → meters via `extent_diag_m`
   - area operators → m² via `extent_area_m2`

5. **Writes outputs once to a shared directory**
   - Prevents redundant computation across experiments
   - Guarantees every experiment uses the **same map representation**
   - Avoids accidental overwrites while keeping artifacts reusable

### Why this is important

- Ensures **consistent normalization** between training and evaluation  
- Provides the necessary per-map reference scales for parameter un-normalization  
- Improves reproducibility and efficiency by reusing identical map embeddings  
- Supports fair comparison between:
  - prompt-only baselines (which ignore map embeddings)
  - fused prompt + map hybrids
  - different prompt backends (USE vs OpenAI)

At the end of this step, the repository contains a self-contained set of map embeddings
ready to be concatenated with prompt embeddings in the next stage.

In [4]:
# ===================== CELL 3 — Map embeddings (shared) =====================

from pathlib import Path
from src.train.run_map_embeddings import run_map_embeddings_from_config

# Map embeddings do NOT depend on the prompt backend, so we compute them once and reuse.
MAP_EMB_DIR = Path(PATHS.MAP_OUT) / "shared_extent"
MAP_EMB_DIR.mkdir(parents=True, exist_ok=True)

# Expected outputs (based on map_embeddings writer)
maps_npz = MAP_EMB_DIR / "maps_embeddings.npz"
maps_pq  = MAP_EMB_DIR / "maps.parquet"

if maps_npz.exists() and maps_pq.exists():
    print("✅ Map embeddings already exist — skipping recomputation.")
    print("   Using:", MAP_EMB_DIR)
    map_meta = {"out_dir": str(MAP_EMB_DIR), "skipped": True}
else:
    print("=== Running map embeddings once (shared) ===")
    print("Output dir:", MAP_EMB_DIR)

    map_meta = run_map_embeddings_from_config(
        maps_root=Path(PATHS.MAPS_ROOT),
        input_pattern=PATHS.INPUT_MAPS_PATTERN,
        user_study_xlsx=Path(PATHS.USER_STUDY_XLSX),
        responses_sheet=PATHS.RESPONSES_SHEET,
        tile_id_col=PATHS.TILE_ID_COL,
        complete_col=PATHS.COMPLETE_COL,
        remove_col=PATHS.REMOVE_COL,
        only_complete=bool(PATHS.ONLY_COMPLETE),
        exclude_removed=bool(PATHS.EXCLUDE_REMOVED),
        out_dir=MAP_EMB_DIR,
        verbosity=1,
        norm="extent",  # dynamic per-map normalization
    )

    print("✅ Map embeddings completed.")

print(map_meta)

# Expose shared map embedding dir for later cells (concat / training)
print("MAP_EMB_DIR:", MAP_EMB_DIR)


=== Running map embeddings once (shared) ===
Output dir: /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent
✅ Map embeddings completed.
MapEmbeddingRunMeta(n_tiles_allowed=399, n_maps_found=824, n_maps_used=399, max_polygons=653, out_dir='/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent', embeddings_path='/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent/maps_embeddings.npz', maps_parquet_path='/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent/maps.parquet')
MAP_EMB_DIR: /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent


### 🔢 Inferring Embedding Dimensions (Experiment-Aware)

In this step, we **infer embedding dimensionalities directly from the saved embedding files**
rather than relying on configuration defaults. Dimensions are inferred **per experiment** to
account for different prompt embedding backends (e.g., USE vs OpenAI), while map embeddings
are shared across experiments.

This ensures:
- A **single source of truth** for feature dimensions
- **Consistency** between training and evaluation pipelines
- Robustness to changes in embedding models or backend configurations
- Correct handling of mixed feature modes (prompt-only vs. fused prompt + map)

Specifically, we:

- Load **prompt embeddings** from  
  `PATHS.PROMPT_OUT/<experiment_name>/prompts_embeddings.npz`
- Load **map embeddings** from the shared map embedding directory  
  `PATHS.MAP_OUT/<shared_folder>/maps_embeddings.npz`
- Infer dimensions as follows:
  - `PROMPT_DIM` — from prompt embeddings (per experiment)
  - `MAP_DIM` — from map embeddings (shared)
  - `FUSED_DIM` — computed per experiment:
    - `prompt_only` → `PROMPT_DIM`
    - `fused` → `MAP_DIM + PROMPT_DIM`

If inferred dimensions differ from those defined in the global configuration (`CFG`),
the inferred values take precedence for all downstream processing.

The inferred dimensions are stored in the experiment registry and used consistently by:
- feature preprocessing
- operator classification
- parameter regression
- evaluation and inference

This guarantees that all downstream models operate on **correctly shaped feature vectors**
and that comparisons between experiments remain valid and reproducible.


In [5]:
# ===================== CELL 4 — Infer embedding dimensions (multi-experiment) =====================

from pathlib import Path
import numpy as np

def _infer_dim_from_npz(npz_path: Path) -> int:
    if not npz_path.exists():
        raise FileNotFoundError(f"Missing embeddings file: {npz_path}")
    z = np.load(npz_path, allow_pickle=True)
    if "E" not in z:
        raise ValueError(f"{npz_path} missing array 'E'")
    E = z["E"]
    if E.ndim != 2 or E.shape[1] <= 0:
        raise ValueError(f"Invalid embedding matrix in {npz_path}: shape={E.shape}")
    return int(E.shape[1])

# -------------------------------
# Map dim (shared across all experiments)
# -------------------------------
maps_npz = Path(MAP_EMB_DIR) / "maps_embeddings.npz"
MAP_DIM_INF = _infer_dim_from_npz(maps_npz)

print("✅ Inferred MAP_DIM from shared maps:", MAP_DIM_INF)

# -------------------------------
# Prompt dim per experiment + fused dim per experiment
# -------------------------------
dims_by_experiment = {}

for exp_name, cfg in EXPERIMENTS.items():
    prm_npz = Path(PATHS.PROMPT_OUT) / exp_name / "prompts_embeddings.npz"
    PROMPT_DIM_INF = _infer_dim_from_npz(prm_npz)

    feature_mode = cfg["feature_mode"]
    if feature_mode == "prompt_only":
        fused_dim = PROMPT_DIM_INF
        map_dim = 0
    elif feature_mode == "fused":
        fused_dim = MAP_DIM_INF + PROMPT_DIM_INF
        map_dim = MAP_DIM_INF
    else:
        raise ValueError(f"Unknown feature_mode for {exp_name}: {feature_mode}")

    # store into cfg for later cells
    cfg["map_dim"] = int(map_dim)
    cfg["prompt_dim"] = int(PROMPT_DIM_INF)
    cfg["fused_dim"] = int(fused_dim)

    dims_by_experiment[exp_name] = {
        "feature_mode": feature_mode,
        "MAP_DIM": int(map_dim),
        "PROMPT_DIM": int(PROMPT_DIM_INF),
        "FUSED_DIM": int(fused_dim),
    }

# Print summary
print("\n✅ Inferred dims per experiment:")
for exp_name, d in dims_by_experiment.items():
    print(f" - {exp_name:12s} | mode={d['feature_mode']:10s} | MAP_DIM={d['MAP_DIM']:4d} | PROMPT_DIM={d['PROMPT_DIM']:4d} | FUSED_DIM={d['FUSED_DIM']:4d}")

# Optional: warn if global CFG is stale (informational only)
if MAP_DIM_INF != int(CFG.MAP_DIM):
    print("\n⚠️ CFG.MAP_DIM differs from inferred MAP_DIM (using inferred for experiments).")
    print(f"   inferred MAP_DIM={MAP_DIM_INF} vs CFG.MAP_DIM={CFG.MAP_DIM}")


✅ Inferred MAP_DIM from shared maps: 165

✅ Inferred dims per experiment:
 - prompt_only  | mode=prompt_only | MAP_DIM=   0 | PROMPT_DIM= 512 | FUSED_DIM= 512
 - use_map      | mode=fused      | MAP_DIM= 165 | PROMPT_DIM= 512 | FUSED_DIM= 677
 - openai_map   | mode=fused      | MAP_DIM= 165 | PROMPT_DIM=1536 | FUSED_DIM=1701


### 🔗 Feature Construction (Prompt-Only vs. Fused Map + Prompt Embeddings)

In this step, we construct the final feature matrices used for training and evaluation by
aligning prompt embeddings with map embeddings and exporting **experiment-scoped** artifacts.

#### What this step does (per experiment)

- Loads **prompt embeddings** from  
  `PATHS.PROMPT_OUT/<experiment_name>/prompts_embeddings.npz`
- Loads **map embeddings** from the shared map embedding directory  
  `PATHS.MAP_OUT/<shared_folder>/maps_embeddings.npz`
- Aligns samples via the authoritative pairing table (`prompts.parquet` → `map_id/tile_id` + `prompt_id`)
- Merges **dynamic map extent metadata** (e.g., `extent_diag_m`, `extent_area_m2`) from `maps.parquet`
  so downstream regression can convert normalized parameters into real-world units.

#### Feature modes supported

- **`prompt_only`**  
  Uses only prompt vectors:  
  `X = prompt_embedding`

- **`fused`**  
  Concatenates map and prompt vectors:  
  `X = [map_embedding | prompt_embedding]`

#### Outputs written per experiment

All artifacts are saved into the experiment’s `train_out` directory (to prevent overwrites):

- `X_prompt.npy` or `X_concat.npy` — final feature matrix (depending on feature mode)
- `train_pairs.parquet` — aligned metadata (including `operator`, `param_value`, and extent references)
- `meta.json` — provenance (sources, options) and shape information

#### Why this design

- Keeps **training and evaluation perfectly aligned** by exporting a single, consistent pairing table
- Avoids hard-coded dimensions by relying on the saved embedding files
- Supports **multiple experiments side-by-side** without overwriting artifacts
- Enables **dynamic extent-aware** parameter regression (meters / m² scaling) downstream
- Ensures fair comparison: prompt-only baselines vs. fused map+prompt models

After this step, each experiment has a complete, self-contained dataset ready for:
1. Operator classification  
2. Per-operator parameter regression  
3. End-to-end evaluation in the evaluation notebook


In [7]:
# ===================== CELL 5 — Concatenate embeddings (multi-experiment) =====================

from pathlib import Path
from src.train.run_concat_features import run_concat_features_from_dirs

print("\n=== Concatenating features for all experiments ===")

concat_meta_by_experiment = {}

def _to_concat_mode(feature_mode: str) -> str:
    """
    Map our notebook naming to what run_concat_features_from_dirs expects.
    """
    fm = str(feature_mode).strip().lower()
    if fm in {"prompt_only"}:
        return "prompt_only"
    if fm in {"fused", "prompt_plus_map"}:
        return "prompt_plus_map"
    raise ValueError(f"Unsupported feature_mode in EXPERIMENTS: {feature_mode}")

for exp_name, cfg in EXPERIMENTS.items():

    feature_mode = cfg["feature_mode"]
    mode_for_concat = _to_concat_mode(feature_mode)

    train_out_dir = Path(cfg["train_out"])
    train_out_dir.mkdir(parents=True, exist_ok=True)

    # Experiment-specific prompt embeddings
    prompt_out_dir = Path(PATHS.PROMPT_OUT) / exp_name

    # Shared map embeddings (from Cell 3)
    map_out_dir = MAP_EMB_DIR

    print(f"\n🧪 Experiment: {exp_name}")
    print(f"   Feature mode (cfg)   : {feature_mode}")
    print(f"   Feature mode (concat): {mode_for_concat}")
    print(f"   Prompt out dir       : {prompt_out_dir}")
    print(f"   Map out dir          : {map_out_dir}")
    print(f"   Train out dir        : {train_out_dir}")

    meta = run_concat_features_from_dirs(
        prompt_out_dir=prompt_out_dir,
        map_out_dir=map_out_dir,
        out_dir=train_out_dir,
        exp_name=exp_name,
        feature_mode=mode_for_concat,   # must be "prompt_only" or "prompt_plus_map"
        verbosity=1,
    )

    concat_meta_by_experiment[exp_name] = meta
    print("   ✅ Concatenation completed.")

print("\n✅ All concatenations finished.")


=== Concatenating features for all experiments ===

🧪 Experiment: prompt_only
   Feature mode (cfg)   : prompt_only
   Feature mode (concat): prompt_only
   Prompt out dir       : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/prompt_only
   Map out dir          : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent
   Train out dir        : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_prompt_only
   ✅ Concatenation completed.

🧪 Experiment: use_map
   Feature mode (cfg)   : fused
   Feature mode (concat): prompt_plus_map
   Prompt out dir       : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/use_map
   Map out dir          : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/shared_extent
   Train out dir        : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_use
   ✅ Concatenation completed.

🧪 Experiment: openai_map
   Feature mode (cfg)   : fused
   Feature

## Step 6 — Load Training Data and Build Normalized Regression Target (Experiment-Aware)

In this step, we load the **experiment-specific training data** produced by the feature
construction stage and build the final learning targets for both classification and regression.

Because this notebook runs **multiple experiments**, the same procedure is applied
**independently for each experiment**, using its own `train_out` directory.

### What happens here (per experiment)

1. **Load the feature matrix**
   - `prompt_only` experiments load prompt-only features (e.g., `X_prompt.npy`)
   - fused experiments load concatenated features (e.g., `X_concat.npy`)

2. **Load the paired metadata table**
   - `train_pairs.parquet` containing aligned `(map_id, prompt_id)` rows and dynamic extent references

3. **Attach labels and apply consistent filtering**
   - Ensures only valid user study rows are included (e.g., `complete == True`, `remove == False`)
   - Ensures `operator` and `param_value` are present (and prompt text if required)

4. **Validate dynamic extent references**
   - Confirms the presence of per-map reference scales required for normalization:
     - `extent_diag_m`
     - `extent_area_m2`

5. **Compute the normalized regression target `param_norm`**
   Normalization depends on the operator group:

   - **Distance-based operators** (`aggregate`, `displace`, `simplify`):  
     `param_norm = param_value / extent_diag_m`

   - **Area-based operators** (`select`):  
     `param_norm = param_value / extent_area_m2`

### Why this normalization is used

This step converts parameters from heterogeneous, map-scale-dependent units into a
scale-aware normalized target. It allows per-operator regressors to generalize across
maps of different extents while preserving physical meaning during inference
(when `param_norm` is converted back to meters or m² using the same extent references).

### Outputs

For each experiment, we obtain:

- `X` — feature matrix aligned with labels  
- `df` — cleaned metadata table including `operator`, `param_value`, `param_norm`, and extent references  

These outputs feed directly into the subsequent training stages:
1. Operator classification  
2. Per-operator parameter regression  
3. End-to-end evaluation across experiments


In [8]:
# ===================== CELL 6 — Load training data + compute param_norm (multi-experiment) =====================

from dataclasses import replace
from pathlib import Path

from src.train.load_training_data import load_training_data_with_dynamic_param_norm

def _to_loader_mode(feature_mode: str) -> str:
    """
    Map our notebook naming to what load_training_data_with_dynamic_param_norm expects.
    """
    fm = str(feature_mode).strip().lower()
    if fm == "prompt_only":
        return "prompt_only"
    if fm in {"fused", "prompt_plus_map"}:
        return "prompt_plus_map"
    raise ValueError(f"Unsupported feature_mode in EXPERIMENTS: {feature_mode}")

TRAIN_DATA = {}

print("\n=== Loading training data for all experiments ===")

for exp_name, cfg in EXPERIMENTS.items():
    train_out_dir = Path(cfg["train_out"])
    if not train_out_dir.exists():
        raise FileNotFoundError(f"Missing train_out for {exp_name}: {train_out_dir}")

    feature_mode_loader = _to_loader_mode(cfg["feature_mode"])

    # Many of your helpers read from PATHS.TRAIN_OUT.
    # So we create an experiment-scoped PATHS that points TRAIN_OUT to this experiment folder.
    PATHS_EXP = replace(PATHS, TRAIN_OUT=train_out_dir)

    print(f"\n🧪 Experiment: {exp_name}")
    print(f"   train_out : {train_out_dir}")
    print(f"   mode      : {cfg['feature_mode']}  -> loader mode: {feature_mode_loader}")

    data = load_training_data_with_dynamic_param_norm(
        exp_name=exp_name,
        feature_mode=feature_mode_loader,
        paths=PATHS_EXP,
        cfg=CFG,
        distance_ops=DISTANCE_OPS,
        area_ops=AREA_OPS,
        require_text=True,
    )

    X = data.X
    df = data.df

    print(f"   ✅ Loaded: X={X.shape} | df={df.shape}")
    print("   Operators:", sorted(df[PATHS.OPERATOR_COL].dropna().unique().tolist()))

    TRAIN_DATA[exp_name] = {"X": X, "df": df, "paths": PATHS_EXP}

# Quick preview (optional)
first_key = next(iter(TRAIN_DATA.keys()))
display(TRAIN_DATA[first_key]["df"].head(10))



=== Loading training data for all experiments ===

🧪 Experiment: prompt_only
   train_out : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_prompt_only
   mode      : prompt_only  -> loader mode: prompt_only
   ✅ Loaded: X=(562, 512) | df=(562, 15)
   Operators: ['aggregate', 'displace', 'select', 'simplify']

🧪 Experiment: use_map
   train_out : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_use
   mode      : fused  -> loader mode: prompt_plus_map
   ✅ Loaded: X=(562, 677) | df=(562, 15)
   Operators: ['aggregate', 'displace', 'select', 'simplify']

🧪 Experiment: openai_map
   train_out : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out_openai
   mode      : fused  -> loader mode: prompt_plus_map
   ✅ Loaded: X=(562, 1701) | df=(562, 15)
   Operators: ['aggregate', 'displace', 'select', 'simplify']


,map_id,prompt_id,text,extent_diag_m,extent_area_m2,extent_width_m,extent_height_m,extent_minx,extent_miny,extent_maxx,extent_maxy,operator,param_value,intensity,param_norm
0,1304,r00000000,Union few of the buildings.,582.418016,169604.830164,412.352091,411.310707,369009.126498,5.624443e+06,369421.478589,5.624855e+06,aggregate,0.000,medium,0.000000
1,1457,r00000002,Remove small buildings and eliminate narrow an...,496.278103,116190.319796,404.903965,286.957723,370209.074685,5.626969e+06,370613.978650,5.627256e+06,select,17.805,low,0.000153
2,1663,r00000005,Bundle nearby buildings into larger blocks.,533.109561,138157.356917,329.923688,418.755494,371811.357509,5.630840e+06,372141.281197,5.631259e+06,aggregate,0.000,medium,0.000000
3,1122,r00000006,Simplify small geometric details below a speci...,597.176996,178021.909966,434.102877,410.091523,367409.757832,5.630048e+06,367843.860710,5.630458e+06,simplify,1.000,low,0.001675
4,1706,r00000009,Eliminate repeated blocks.,465.952069,108172.503069,315.345720,343.028290,372305.411445,5.628541e+06,372620.757165,5.628884e+06,select,18.226,low,0.000168
5,1174,r00000010,Make a space between the polygons.,485.597704,105023.121090,414.108392,253.612636,367806.423992,5.631230e+06,368220.532385,5.631484e+06,displace,1.975,low,0.004067
6,0116,r00000011,Aggregate some of the buildings.,577.475387,166688.512862,403.286009,413.325801,359417.860505,5.619642e+06,359821.146514,5.620056e+06,aggregate,0.000,medium,0.000000
7,0417,r00000013,Exclude shapes with an area less than 20 squar...,571.706715,163423.957488,404.661369,403.853617,361813.128457,5.622451e+06,362217.789827,5.622855e+06,select,20.000,low,0.000122
8,1488,r00000014,Aggregate the blocks.,500.054694,115279.716486,276.818974,416.444418,370607.033823,5.619641e+06,370883.852797,5.620058e+06,aggregate,0.000,medium,0.000000
9,0175,r00000015,Simplify the shapes of the small buildings by ...,503.061104,119618.905293,409.629591,292.017247,359816.877837,5.623767e+06,360226.507427,5.624059e+06,aggregate,0.000,medium,0.000000


## Step 7 — Train/Validation/Test Split (Shared, Leakage-Free by Map)

In this step, we construct a **reproducible and fair** train/validation/test split that is
**shared across all experiments** (prompt-only, USE + map, OpenAI + map).  
The split is computed **once** and then applied to every experiment to ensure that
performance differences are attributable solely to the feature representation.

---

### Constraints enforced

- **No leakage by map**  
  The same `map_id` never appears in more than one split (train / validation / test).

- **Multi-prompt maps are forced into TRAIN**  
  If a map has multiple prompts, *all* corresponding samples are assigned to the
  training set.  
  As a result, validation and test sets contain **single-prompt maps only**.

- **Consistency across experiments**  
  The exact same samples (identified by `(map_id, prompt_id)`) are used for
  train/validation/test in every experiment.

---

### Stratification strategy

To obtain balanced splits while respecting the above constraints, stratification is applied as:

- Primary: `operator × intensity` (if sufficient samples exist), otherwise
- Fallback: `operator` only (automatically selected if finer stratification is infeasible)

---

### Coverage requirement

Each split is required to contain **all operators** in the fixed class set:

- `simplify`
- `select`
- `aggregate`
- `displace`

This guarantees that classification and regression models can be trained and evaluated
for every operator.

---

### Outputs

- A single shared split definition is saved to disk as:  
  `splits_shared.json`
- For each experiment, the split is applied to slice:
  - `X` — the feature matrix
  - `df` — the aligned metadata table

The resulting subsets (`train`, `val`, `test`) are then used in all downstream
training and evaluation steps.

---

This design ensures:
- **Leakage-free evaluation**
- **Fair, apples-to-apples comparison** between experiments
- **Reproducibility**, since the split is deterministic and persisted to disk


In [9]:
# ===================== CELL 7 — Shared Train/Val/Test Split (fair across experiments) =====================

from pathlib import Path
import pandas as pd

from src.train.splitting import make_splits_multi_prompt_to_train

FIXED_CLASSES = ["simplify", "select", "aggregate", "displace"]
USE_INTENSITY_FOR_STRAT = True

OP_COL  = PATHS.OPERATOR_COL
INT_COL = PATHS.INTENSITY_COL

# Where to save ONE shared split (used by all experiments)
SPLITS_DIR = Path(PATHS.SPLIT_OUT)
SPLITS_DIR.mkdir(parents=True, exist_ok=True)
split_path = SPLITS_DIR / "splits_shared.json"

# -------------------------------
# Choose a reference experiment to compute the split once
# (any experiment works as long as df has same rows)
# -------------------------------
ref_exp = next(iter(TRAIN_DATA.keys()))
ref_df = TRAIN_DATA[ref_exp]["df"].copy()
ref_X  = TRAIN_DATA[ref_exp]["X"]

# Build a stable key per row for mapping splits across experiments
# Assumes train_pairs.parquet contains map_id and prompt_id
if not {"map_id", "prompt_id"}.issubset(ref_df.columns):
    raise ValueError("Expected columns {'map_id','prompt_id'} in df (from train_pairs.parquet).")

ref_df["row_key"] = ref_df["map_id"].astype(str).str.zfill(4) + "::" + ref_df["prompt_id"].astype(str)

# Compute split ONCE
print(f"\n=== Computing shared split using reference experiment: {ref_exp} ===")

split = make_splits_multi_prompt_to_train(
    df=ref_df,
    X=ref_X,
    op_col=OP_COL,
    intensity_col=INT_COL if INT_COL in ref_df.columns else None,
    map_id_col="map_id",
    fixed_classes=FIXED_CLASSES,
    use_intensity_for_strat=USE_INTENSITY_FOR_STRAT,
    seed=int(CFG.SEED),
    val_ratio=float(CFG.VAL_RATIO),
    test_ratio=float(CFG.TEST_RATIO),
    max_attempts=500,
    save_splits_json=split_path,
    verbose=True,
)

train_idx_ref, val_idx_ref, test_idx_ref = split.train_idx, split.val_idx, split.test_idx

# Convert indices -> row_key sets (this is what we transfer across experiments)
train_keys = set(ref_df.loc[train_idx_ref, "row_key"].tolist())
val_keys   = set(ref_df.loc[val_idx_ref,   "row_key"].tolist())
test_keys  = set(ref_df.loc[test_idx_ref,  "row_key"].tolist())

# Sanity: no overlap
assert train_keys.isdisjoint(val_keys) and train_keys.isdisjoint(test_keys) and val_keys.isdisjoint(test_keys)

print("\n✅ Shared split created:")
print(f"   Train keys: {len(train_keys)} | Val keys: {len(val_keys)} | Test keys: {len(test_keys)}")
print(f"   Saved to: {split_path}")

# -------------------------------
# Apply the SAME split to every experiment by mapping row_key -> indices
# -------------------------------
SPLITS = {}  # exp_name -> dict with X_train/X_val/X_test and df_train/df_val/df_test

for exp_name, pack in TRAIN_DATA.items():
    df = pack["df"].copy()
    X  = pack["X"]

    df["row_key"] = df["map_id"].astype(str).str.zfill(4) + "::" + df["prompt_id"].astype(str)

    # Build index arrays in the current df order
    train_idx = df.index[df["row_key"].isin(train_keys)].to_numpy()
    val_idx   = df.index[df["row_key"].isin(val_keys)].to_numpy()
    test_idx  = df.index[df["row_key"].isin(test_keys)].to_numpy()

    # Check that every key exists in this experiment (should be identical)
    missing = (train_keys | val_keys | test_keys) - set(df["row_key"].tolist())
    if missing:
        raise ValueError(
            f"Experiment '{exp_name}' is missing {len(missing)} rows from the shared split "
            f"(first few: {list(sorted(missing))[:5]}). "
            "This usually means train_pairs.parquet differs between experiments."
        )

    X_train, X_val, X_test = X[train_idx], X[val_idx], X[test_idx]
    df_train = df.loc[train_idx].reset_index(drop=True)
    df_val   = df.loc[val_idx].reset_index(drop=True)
    df_test  = df.loc[test_idx].reset_index(drop=True)

    SPLITS[exp_name] = {
        "train_idx": train_idx,
        "val_idx": val_idx,
        "test_idx": test_idx,
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "df_train": df_train, "df_val": df_val, "df_test": df_test,
    }

    print(f"\n🧪 {exp_name}")
    print("Rows -> Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)



=== Computing shared split using reference experiment: prompt_only ===
=== DATASET SUMMARY ===
Total rows (prompts): 562
Unique maps: 399
Multi-prompt maps (>1 prompt): 22
Single-prompt maps (=1 prompt): 377

Top 10 maps by prompt count:
map_id
1646    30
1304    29
1755    26
1532    13
0127    10
0168     8
0142     7
0078     6
0080     6
0001     6
dtype: int64

=== SPLIT SUMMARY ===
✅ Split found (seed=42)
Train maps: 285  (includes multi-prompt maps: 22)
Val maps:   57
Test maps:  57
✅ Verified: no map_id leakage across splits.
✅ Verified: all multi-prompt maps are in TRAIN.

TRAIN — Operator counts
operator
select       144
aggregate    134
simplify     109
displace      61
Name: count, dtype: Int64

VAL — Operator counts
operator
select       19
aggregate    16
simplify     13
displace      9
Name: count, dtype: Int64

TEST — Operator counts
operator
select       19
aggregate    16
simplify     13
displace      9
Name: count, dtype: Int64

TRAIN — Operator × Intensity table (c

## Step 8 — Modality-Aware Preprocessing (Experiment-Aware)

This step applies preprocessing tailored to the input modalities **separately for each
experiment**, using **training data only** to fit preprocessing parameters.  
A preprocessing bundle is saved per experiment so the exact same transformations can be
reused during evaluation and inference.

---

### Prompt embeddings (all feature modes)

Prompt vectors are normalized using **row-wise L2 normalization** to ensure consistent scale
across samples and embedding backends (e.g., USE vs OpenAI).

---

### Map embeddings (only for fused prompt + map experiments)

When the feature mode includes map vectors (`prompt_plus_map` / fused), the map block is
processed using a robust pipeline:

1. Replace non-finite values (`±inf`) with `NaN`
2. Impute missing values using the **median** (fit on training data only)
3. Clip each feature to training-set **5th–95th percentiles** to reduce outlier impact
4. Drop zero-variance (or near-constant) features based on training data
5. Apply **RobustScaler** using quantile range **(5, 95)**

The prompt block remains L2-normalized and is concatenated with the processed map block.

---

### Outputs (per experiment)

For each experiment we obtain:

- `X_train_s`, `X_val_s`, `X_test_s` — preprocessed matrices ready for training

A preprocessing bundle is saved into the experiment’s model output folder as:

- `preproc.joblib`

This ensures the exact same preprocessing can be reused for:
- reproducible training
- consistent evaluation across experiments
- deployment-time inference (operator + parameter prediction)


In [10]:
# ===================== CELL 8 — Modality-aware preprocessing (per experiment) =====================

from pathlib import Path
from src.train.preprocessing import fit_transform_modality_preproc

PREPROC = {}  # exp_name -> dict with scaled arrays + bundle path

print("\n=== Fitting modality-aware preprocessing per experiment ===")

def _to_preproc_mode(feature_mode: str) -> str:
    """
    Map notebook feature naming to preprocessing expectations.
    """
    fm = str(feature_mode).strip().lower()
    if fm == "prompt_only":
        return "prompt_only"
    if fm in {"fused", "prompt_plus_map"}:
        return "prompt_plus_map"
    raise ValueError(f"Unsupported feature_mode for preprocessing: {feature_mode}")

for exp_name, cfg in EXPERIMENTS.items():

    split = SPLITS[exp_name]
    feature_mode = cfg["feature_mode"]
    preproc_mode = _to_preproc_mode(feature_mode)

    map_dim    = cfg["map_dim"]
    prompt_dim = cfg["prompt_dim"]

    model_out_dir = Path(cfg["model_out"])
    model_out_dir.mkdir(parents=True, exist_ok=True)

    preproc_path = model_out_dir / "preproc.joblib"

    print(f"\n🧪 Experiment: {exp_name}")
    print(f"   Feature mode : {feature_mode} -> {preproc_mode}")
    print(f"   map_dim      : {map_dim}")
    print(f"   prompt_dim   : {prompt_dim}")
    print(f"   Save preproc : {preproc_path}")

    res = fit_transform_modality_preproc(
        X_train=split["X_train"],
        X_val=split["X_val"],
        X_test=split["X_test"],
        feature_mode=preproc_mode,   # "prompt_only" or "prompt_plus_map"
        map_dim=map_dim,
        prompt_dim=prompt_dim,
        eps=1e-12,
        clip_q=(5, 95),
        impute_strategy="median",
        robust_qrange=(5, 95),
        save_path=preproc_path,
    )

    PREPROC[exp_name] = {
        "X_train_s": res.X_train_s,
        "X_val_s":   res.X_val_s,
        "X_test_s":  res.X_test_s,
        "bundle_path": res.bundle_path,
    }

    print("   ✅ Preprocessing complete.")
    print("   Shapes:",
          res.X_train_s.shape,
          res.X_val_s.shape,
          res.X_test_s.shape)

print("\n✅ All preprocessing finished.")



=== Fitting modality-aware preprocessing per experiment ===

🧪 Experiment: prompt_only
   Feature mode : prompt_only -> prompt_only
   map_dim      : 0
   prompt_dim   : 512
   Save preproc : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_prompt_only/preproc.joblib
   ✅ Preprocessing complete.
   Shapes: (448, 512) (57, 512) (57, 512)

🧪 Experiment: use_map
   Feature mode : fused -> prompt_plus_map
   map_dim      : 165
   prompt_dim   : 512
   Save preproc : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_use_map/preproc.joblib
   ✅ Preprocessing complete.
   Shapes: (448, 677) (57, 677) (57, 677)

🧪 Experiment: openai_map
   Feature mode : fused -> prompt_plus_map
   map_dim      : 165
   prompt_dim   : 1536
   Save preproc : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_openai_map/preproc.joblib
   ✅ Preprocessing complete.
   Shapes: (448, 1701) (57, 1701) (57, 1701)

✅ All preprocessing finished.


## Step 9 — Build Class Labels and Sample Weights (Experiment-Aware)

In this step, we construct the classification labels and training sample weights **for each
experiment**, using the same fixed class order and the same split definition. This guarantees
that differences in performance across experiments are due to the feature representation,
not label encoding or sampling artifacts.

---

### Fixed class encoding

Operator labels are encoded using a fixed global class order:

`[simplify, select, aggregate, displace]`

This guarantees consistent label indices across:
- training
- saved model bundles
- evaluation and inference code

Because the split is shared across experiments, this encoding remains stable and comparable.

---

### Sample weighting

Training samples are weighted to address two common sources of bias:

1. **Class imbalance**
   - Balanced class weights are computed from the training distribution to prevent majority
     classes from dominating learning.

2. **Map-level prompt multiplicity**
   - Some `map_id`s have multiple prompts.
   - To prevent such maps from contributing disproportionately, each map contributes
     approximately equal total weight by assigning each prompt a map-weight of:

   `map_weight = 1 / (#prompts for that map_id)`

---

### Final weight definition

The final per-sample weight used during training is:

`sample_w = class_weight(operator) × map_weight(map_id)`

These weights are used during classifier training (and optionally regression training)
to improve robustness and ensure fair learning across operators and maps.


In [11]:
# ===================== CELL 9 — Build labels + sample weights (per experiment) =====================

import numpy as np
from src.train.labels_and_weights import build_labels_and_sample_weights

OP_COL = PATHS.OPERATOR_COL  # usually "operator"

LABELS = {}  # exp_name -> labels, weights, class_names, etc.

print("\n=== Building labels and sample weights per experiment ===")

for exp_name, split in SPLITS.items():

    df_train = split["df_train"]
    df_val   = split["df_val"]
    df_test  = split["df_test"]

    lab = build_labels_and_sample_weights(
        df_train=df_train,
        df_val=df_val,
        df_test=df_test,
        op_col=OP_COL,
        map_id_col="map_id",
        fixed_classes=FIXED_CLASSES,
        use_map_weight=True,
        class_weight_mode="balanced",
    )

    class_names = np.array(lab.class_names)

    LABELS[exp_name] = {
        "class_names": class_names,
        "y_train_cls": lab.y_train,
        "y_val_cls":   lab.y_val,
        "y_test_cls":  lab.y_test,
        "sample_w":    lab.sample_w,
        "class_weight_map": lab.class_weight_map,
    }

    print(f"\n🧪 {exp_name}")
    print("Classes (fixed order):", list(class_names))
    print("Class weights:", lab.class_weight_map)
    print(
        "y_train/y_val/y_test shapes:",
        lab.y_train.shape, lab.y_val.shape, lab.y_test.shape
    )
    sw = lab.sample_w
    print("Sample weight summary:", {"min": float(sw.min()), "max": float(sw.max()), "mean": float(sw.mean())})

# Optional: sanity check that class order is identical across experiments
first = next(iter(LABELS.keys()))
base_classes = LABELS[first]["class_names"].tolist()
for exp_name in LABELS.keys():
    if LABELS[exp_name]["class_names"].tolist() != base_classes:
        raise ValueError(f"Class order differs in experiment {exp_name}. This would break fair comparison.")
print("\n✅ Label build complete for all experiments (class order consistent).")



=== Building labels and sample weights per experiment ===

🧪 prompt_only
Classes (fixed order): [np.str_('simplify'), np.str_('select'), np.str_('aggregate'), np.str_('displace')]
Class weights: {'simplify': 1.0275229357798166, 'select': 0.7777777777777778, 'aggregate': 0.835820895522388, 'displace': 1.8360655737704918}
y_train/y_val/y_test shapes: (448,) (57,) (57,)
Sample weight summary: {'min': 0.025925925925925925, 'max': 1.8360655737704918, 'mean': 0.6487687942076353}

🧪 use_map
Classes (fixed order): [np.str_('simplify'), np.str_('select'), np.str_('aggregate'), np.str_('displace')]
Class weights: {'simplify': 1.0275229357798166, 'select': 0.7777777777777778, 'aggregate': 0.835820895522388, 'displace': 1.8360655737704918}
y_train/y_val/y_test shapes: (448,) (57,) (57,)
Sample weight summary: {'min': 0.025925925925925925, 'max': 1.8360655737704918, 'mean': 0.6487687942076353}

🧪 openai_map
Classes (fixed order): [np.str_('simplify'), np.str_('select'), np.str_('aggregate'), np.st

## Step 10 — Operator Classification Model (MLP, Experiment-Aware)

This step trains the operator classifier that predicts one of the four map generalization
operators:

`{simplify, select, aggregate, displace}`

The same training protocol is applied **independently for each experiment** (prompt-only,
USE + map, OpenAI + map) using the **shared split**. This ensures that differences in
performance across experiments are attributable to the feature representation rather than
changes in training procedure.

---

### Model and training strategy

- We use an **MLPClassifier** (multi-layer perceptron).
- Hyperparameters are explored via a lightweight random search over:
  - hidden layer sizes
  - weight decay (`alpha`)
  - learning rate schedule
  - batch size / optimization settings (as implemented in the helper)

---

### Validation protocol (leakage-free)

To prevent leakage, we perform **grouped cross-validation** using `map_id`:

- prompts from the same map are never split across folds

This is critical because multiple prompts may refer to the same map and would otherwise
inflate performance due to memorization.

---

### Model selection and evaluation

The best configuration is selected using validation performance (with grouped CV used for
reliable hyperparameter tuning). The selected model is then retrained on the full training
split and evaluated on validation and test splits.

---

### Outputs (per experiment)

For each experiment, the trained classifier is saved into the experiment’s model folder as:

- `classifier.joblib`

This classifier is later used to:
1. predict the operator class
2. route each sample to the correct operator-specific parameter regressor


In [ ]:
# ===================== CELL 10 — Train classifier (per experiment, MLP search + final fit) =====================

from pathlib import Path
import numpy as np
from dataclasses import asdict, is_dataclass

from src.train.train_classifier import train_mlp_classifier_with_search

CLF_RESULTS = {}  # exp_name -> ClassifierTrainResult

def _safe_get(obj, *names, default=None):
    for n in names:
        if hasattr(obj, n):
            return getattr(obj, n)
    return default

print("\n=== Training operator classifiers for all experiments ===")

for exp_name, cfg in EXPERIMENTS.items():

    split = SPLITS[exp_name]
    pre   = PREPROC[exp_name]
    lab   = LABELS[exp_name]

    X_train_s = pre["X_train_s"]
    X_val_s   = pre["X_val_s"]
    X_test_s  = pre["X_test_s"]

    y_train = lab["y_train_cls"]
    y_val   = lab["y_val_cls"]
    y_test  = lab["y_test_cls"]
    sample_w = lab["sample_w"]

    class_names = [str(x) for x in lab["class_names"]]

    # Grouped CV: group by map_id to avoid leakage across folds
    groups_tr = split["df_train"]["map_id"].astype(str).to_numpy()

    model_out_dir = Path(cfg["model_out"])
    model_out_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n🧪 Experiment: {exp_name}")
    print(f"   Classes   : {class_names}")
    print(f"   Train X   : {X_train_s.shape}")
    print(f"   Val X     : {X_val_s.shape}")
    print(f"   Test X    : {X_test_s.shape}")
    print(f"   Model out : {model_out_dir}")

    res_clf = train_mlp_classifier_with_search(
        exp_name=exp_name,
        X_train=X_train_s,
        y_train=y_train,
        groups_train=groups_tr,
        sample_w=sample_w,
        X_val=X_val_s,
        y_val=y_val,
        X_test=X_test_s,
        y_test=y_test,
        class_names=class_names,
        out_dir=model_out_dir,
        n_iter=50,
        n_splits=5,
        seed=int(CFG.SEED),
        verbose=True,
        save_name="classifier.joblib",
    )

    CLF_RESULTS[exp_name] = res_clf

    # ---- robust reporting (no assumptions about field names) ----
    model_path = _safe_get(res_clf, "model_path", "path", default=str(model_out_dir / "classifier.joblib"))
    best_val_f1 = _safe_get(res_clf, "best_val_f1", "val_f1", "best_f1", default=None)
    best_val_acc = _safe_get(res_clf, "best_val_acc", "val_acc", "best_accuracy", default=None)
    test_f1 = _safe_get(res_clf, "test_f1", default=None)
    test_acc = _safe_get(res_clf, "test_acc", "accuracy_test", default=None)

    print("   ✅ Classifier training done.")
    print("   Saved to:", model_path)
    if best_val_f1 is not None or best_val_acc is not None:
        print("   Best VAL:", {"macro_f1": best_val_f1, "acc": best_val_acc})
    if test_f1 is not None or test_acc is not None:
        print("   TEST     :", {"macro_f1": test_f1, "acc": test_acc})

    # Optional: print available fields once for debugging
    if exp_name == next(iter(EXPERIMENTS.keys())):
        if is_dataclass(res_clf):
            print("   (debug) Result fields:", list(asdict(res_clf).keys()))
        else:
            print("   (debug) Result attrs :", [a for a in dir(res_clf) if not a.startswith("_")])

print("\n✅ All classifiers trained.")


=== Training operator classifiers for all experiments ===

🧪 Experiment: prompt_only
   Classes   : ['simplify', 'select', 'aggregate', 'displace']
   Train X   : (448, 512)
   Val X     : (57, 512)
   Test X    : (57, 512)
   Model out : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models/exp_prompt_only

Searching 50 MLP configs...
[01/50] cvF1=0.809±0.048 | VAL F1=0.944 acc=0.947 | (128, 64), α=2.02e-02, lr=1.2e-03, bs=16
[02/50] cvF1=0.813±0.059 | VAL F1=0.944 acc=0.947 | (256, 128), α=3.49e-05, lr=1.7e-04, bs=64
[03/50] cvF1=0.808±0.064 | VAL F1=0.944 acc=0.947 | (256,), α=1.03e-02, lr=7.7e-04, bs=128
[04/50] cvF1=0.804±0.066 | VAL F1=0.944 acc=0.947 | (256,), α=1.18e-05, lr=2.7e-03, bs=128
[05/50] cvF1=0.806±0.058 | VAL F1=0.944 acc=0.947 | (256, 128, 64), α=5.47e-05, lr=1.9e-04, bs=16
[06/50] cvF1=0.804±0.066 | VAL F1=0.944 acc=0.947 | (64,), α=1.14e-04, lr=6.0e-04, bs=128
[07/50] cvF1=0.800±0.062 | VAL F1=0.944 acc=0.947 | (64,), α=1.03e-04, lr=8.0e-04, bs=32
[08/5

/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/

[23/50] cvF1=0.817±0.054 | VAL F1=0.944 acc=0.947 | (64,), α=4.84e-03, lr=2.0e-04, bs=128
[24/50] cvF1=0.804±0.066 | VAL F1=0.944 acc=0.947 | (256,), α=4.91e-05, lr=1.1e-03, bs=64
[25/50] cvF1=0.806±0.061 | VAL F1=0.944 acc=0.947 | (64,), α=1.28e-03, lr=2.3e-03, bs=32
[26/50] cvF1=0.808±0.061 | VAL F1=0.944 acc=0.947 | (64,), α=1.52e-02, lr=1.8e-03, bs=128
[27/50] cvF1=0.801±0.060 | VAL F1=0.944 acc=0.947 | (128,), α=2.15e-05, lr=3.5e-04, bs=32
[28/50] cvF1=0.799±0.066 | VAL F1=0.944 acc=0.947 | (256, 128), α=3.44e-03, lr=8.7e-04, bs=64


## Step 11 — Parameter Regression (Per-Operator) and Final Model Bundle (Experiment-Aware)

This step trains **operator-specific regressors** to predict the generalization parameter in a
**scale-independent form**, and then packages all trained components into a single,
experiment-scoped model bundle.

The same procedure is applied **independently for each experiment** (prompt-only,
USE + map, OpenAI + map), using the shared data split and preprocessing pipeline.

---

### Regression target

Regressors are trained on the normalized target `param_norm`, defined as:

- **Distance-based operators** (`simplify`, `aggregate`, `displace`):  
  `param_norm = param_value / extent_diag_m`

- **Area-based operators** (`select`):  
  `param_norm = param_value / extent_area_m2`

This normalization allows each regressor to generalize across maps of different spatial
extent while preserving physical meaning. During inference, predictions are converted back
to real-world units using the same per-map extent references.

---

### Training strategy

- One **MLPRegressor per operator**
- Training data restricted to samples of the corresponding operator
- **Grouped cross-validation** (`GroupKFold`) by `map_id` to prevent spatial leakage
- Hyperparameter optimization via `RandomizedSearchCV` for each operator independently

---

### Final model bundle

For each experiment, the trained components are stored together in a single bundle:

- `cls_plus_regressors.joblib`

This bundle contains:
- the trained operator classifier
- the dictionary of operator-specific regressors
- the fixed class order
- normalization metadata (operator groups and extent columns)

Along with the experiment’s `preproc.joblib`, this bundle is sufficient for the evaluation
notebook to compute:

1. **Classifier-only metrics**  
2. **Regressor-only metrics** (oracle operator routing)  
3. **End-to-end pipeline metrics** (predicted operator routing)

This design keeps evaluation simple, reproducible, and fully decoupled from the training
notebook.


In [ ]:
# ===================== CELL 11 — Train per-operator regressors + save final bundle (per experiment) =====================

from pathlib import Path
import joblib

from src.train.train_regressors import train_regressors_per_operator
from src.train.save_bundle import save_cls_plus_regressors_bundle

BUNDLES = {}     # exp_name -> bundle path
REG_RESULTS = {} # exp_name -> regressor training result

print("\n=== Training per-operator regressors and saving final bundles ===")

for exp_name, cfg in EXPERIMENTS.items():

    split = SPLITS[exp_name]
    pre   = PREPROC[exp_name]
    lab   = LABELS[exp_name]
    res_clf = CLF_RESULTS[exp_name]

    X_train_s = pre["X_train_s"]
    df_train  = split["df_train"]
    y_train_cls = lab["y_train_cls"]
    sample_w = lab["sample_w"]

    # Make sure class_names is list[str] (stable ordering)
    cn = [str(x) for x in lab["class_names"]]

    model_out_dir = Path(cfg["model_out"])
    model_out_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n🧪 Experiment: {exp_name}")
    print(f"   Model out: {model_out_dir}")
    print(f"   Train X  : {X_train_s.shape} | df_train: {df_train.shape}")

    # ---- (1) Train per-operator regressors on TRAIN only ----
    reg_res = train_regressors_per_operator(
        X_train_s=X_train_s,
        df_train=df_train,
        y_train_cls=y_train_cls,
        class_names=cn,
        sample_w=sample_w,
        group_col="map_id",
        target_col="param_norm",
        use_log1p=False,
        n_splits=5,
        n_iter=40,
        random_state=int(CFG.SEED),
        verbose=1,
    )

    REG_RESULTS[exp_name] = reg_res

    # ---- (2) Load the trained classifier model (from Cell 10 output) ----
    clf_pack = joblib.load(Path(res_clf.model_path))
    final_clf = clf_pack["model"]

    # ---- (3) Save combined bundle for evaluation notebook ----
    bundle_res = save_cls_plus_regressors_bundle(
        exp_name=exp_name,
        out_dir=model_out_dir,
        classifier=final_clf,
        regressors_by_class=reg_res.regressors_by_class,
        class_names=cn,
        use_log1p=reg_res.use_log1p,
        cv_summary=reg_res.cv_summary,
        distance_ops=DISTANCE_OPS,
        area_ops=AREA_OPS,
        diag_col="extent_diag_m",
        area_col="extent_area_m2",
        save_name="cls_plus_regressors.joblib",  # fixed name inside each experiment folder
    )

    BUNDLES[exp_name] = bundle_res.bundle_path

    print("   ✅ Saved bundle:", bundle_res.bundle_path)
    print("   ✅ Regressors trained for:", sorted(list(reg_res.regressors_by_class.keys())))

print("\n✅ All bundles saved.")
for k, v in BUNDLES.items():
    print(f" - {k:12s}: {v}")
